In [3]:
import pandas as pd

# Define the data structure with columns and example rows
data = {
    'device id': ['device_1', 'device_2', 'device_3'],
    'batch no': [101, 102, 103],
    'lot no': [1001, 1002, 1003],
    'wavelength': [450, 550, 650],
    'level': [1, 2, 3],
    'calibration issue': ['NO Issue', 'NO Issue', 'NO Issue']
}

# Create a DataFrame
df = pd.DataFrame(data)

# Specify the file path
file_path = 'calibration_data.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(file_path, index=False)

print(f"File '{file_path}' has been created with the specified structure and example data.")


File 'calibration_data.xlsx' has been created with the specified structure and example data.


In [2]:
pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.0 kB 660.6 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/250.0 kB 656.4 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.0 kB 731.4 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.0 kB 898.2 kB/s eta 0:00:01
   ------------------------------- ------ 204.8/250.0 kB 958.4 kB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tkinter import *
from tkinter import ttk
from utils.calibration import *

def on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox):
    device_data[device_name] = [
        level_combobox.get(),
        wavelength_combobox.get(),
        duration_combobox.get()
    ]

def create_buttons_with_dropdowns(frame, button_list, level_options, wavelength_options, duration_options):
    global device_data
    device_data = {}  # Dictionary to store device data
    
    def print_device_data():
        print(device_data)
    
    for row, device_name in enumerate(button_list):
        Button(frame, text=device_name, width=8, height=2).grid(row=row, column=0, padx=10, pady=10)
        
        level_combobox = ttk.Combobox(frame, values=level_options, width=8, height=2)
        level_combobox.grid(row=row, column=1, padx=2, pady=10)
        
        wavelength_combobox = ttk.Combobox(frame, values=wavelength_options, width=8, height=2)
        wavelength_combobox.grid(row=row, column=2, padx=2, pady=10)
        
        duration_combobox = ttk.Combobox(frame, values=duration_options, width=8, height=2)
        duration_combobox.grid(row=row, column=3, padx=2, pady=10)
        
        # Bind the event to the lambda function
        level_combobox.bind('<<ComboboxSelected>>', lambda event, device_name=device_name, level_combobox=level_combobox, wavelength_combobox=wavelength_combobox, duration_combobox=duration_combobox: on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox))
        wavelength_combobox.bind('<<ComboboxSelected>>', lambda event, device_name=device_name, level_combobox=level_combobox, wavelength_combobox=wavelength_combobox, duration_combobox=duration_combobox: on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox))
        duration_combobox.bind('<<ComboboxSelected>>', lambda event, device_name=device_name, level_combobox=level_combobox, wavelength_combobox=wavelength_combobox, duration_combobox=duration_combobox: on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox))
    
    # Add a button to print device data
    print_button = Button(frame, text="Print Device Data", command=print_device_data)
    print_button.grid(row=len(button_list), columnspan=4, pady=10)

root = Tk()
root.title("Device Stability Testing")
root.geometry("810x490")

f1 = Frame(root, bg="gray", width=500, height=490)
f1.pack(side=RIGHT, fill=Y)

button_list = ['Device_1', 'Device_2', 'Device_3', 'Device_4', 'Device_5']
level_options = ['Level_1', 'Level_2', 'Level_3']
wavelength_options = ['Wavelength_1', 'Wavelength_2', 'Wavelength_3']
duration_options = ['Duration_1', 'Duration_2', 'Duration_3']

create_buttons_with_dropdowns(f1, button_list, level_options, wavelength_options, duration_options)

root.mainloop()



In [1]:
def filter_and_save_excel(input_csv = 'data/main.csv', output_excel = 'data/output.xlsx'):
    # Check if the input CSV file exists
    if not os.path.exists(input_csv):
        # If the file doesn't exist, create an Excel file with an error message
        with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
            df_error = pd.DataFrame({'Error': ['Input CSV file not found']})
            df_error.to_excel(writer, sheet_name='Error', index=False)
        return
    
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Filter columns where the header contains "L1" or "L5"
    l1_columns = df.filter(regex=r'\bL1\b', axis=1)
    l5_columns = df.filter(regex=r'\bL5\b', axis=1)
    
    # Sort the columns alphabetically based on the column names
    l1_columns = l1_columns.reindex(sorted(l1_columns.columns), axis=1)
    l5_columns = l5_columns.reindex(sorted(l5_columns.columns), axis=1)
    
    # Create a new Excel writer object
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        # Write the filtered and sorted data to different sheets
        if not l1_columns.empty:
            l1_columns.to_excel(writer, sheet_name='L1 Columns', index=False)
        if not l5_columns.empty:
            l5_columns.to_excel(writer, sheet_name='L5 Columns', index=False)
        
        # If both are empty, add a default sheet to avoid the IndexError
        if l1_columns.empty and l5_columns.empty:
            df_empty = pd.DataFrame({'Message': ['No columns with L1 or L5 found']})
            df_empty.to_excel(writer, sheet_name='Info', index=False)

In [2]:
from tkinter import *
from tkinter import ttk


def on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox):
    device_data[device_name] = [
        level_combobox.get(),
        wavelength_combobox.get(),
        duration_combobox.get()
    ]
    print_to_text_box(f"Device {device_name}: Level - {device_data[device_name][0]}, Wavelength - {device_data[device_name][1]}, Duration - {device_data[device_name][2]}")


def create_buttons_with_dropdowns(frame, button_list, level_options, wavelength_options, duration_options):
    global device_data
    device_data = {}  # Dictionary to store device data
    
    def print_device_data():
        print_to_text_box(device_data)
    
    for row, device_name in enumerate(button_list):
        Button(frame, text=device_name, width=15, height=2).grid(row=row, column=0, padx=10, pady=10)
        
        level_combobox = ttk.Combobox(frame, values=level_options, width=8, height=2)
        level_combobox.grid(row=row, column=1, padx=2, pady=10)
        
        wavelength_combobox = ttk.Combobox(frame, values=wavelength_options, width=8, height=2)
        wavelength_combobox.grid(row=row, column=2, padx=2, pady=10)
        
        duration_combobox = ttk.Combobox(frame, values=duration_options, width=8, height=2)
        duration_combobox.grid(row=row, column=3, padx=2, pady=10)
        
        # Bind the event to the lambda function
        level_combobox.bind('<<ComboboxSelected>>', lambda event, device_name=device_name, level_combobox=level_combobox, wavelength_combobox=wavelength_combobox, duration_combobox=duration_combobox: on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox))
        wavelength_combobox.bind('<<ComboboxSelected>>', lambda event, device_name=device_name, level_combobox=level_combobox, wavelength_combobox=wavelength_combobox, duration_combobox=duration_combobox: on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox))
        duration_combobox.bind('<<ComboboxSelected>>', lambda event, device_name=device_name, level_combobox=level_combobox, wavelength_combobox=wavelength_combobox, duration_combobox=duration_combobox: on_selection_change(event, device_name, level_combobox, wavelength_combobox, duration_combobox))
    
    # Add a button to print device data
    print_button = Button(frame, text="Print Device Data", command=print_device_data)
    print_button.grid(row=len(button_list), columnspan=4, pady=10)


def print_to_text_box(text):
    text_box.config(state=NORMAL)
    text_box.insert(END, str(text) + "\n")
    text_box.config(state=DISABLED)
    text_box.see(END)


if __name__ == "__main__":
    root = Tk()
    root.title("Main Window")
    
    main_frame = Frame(root)
    main_frame.pack(padx=10, pady=10)
    
    devices = ["Device 1", "Device 2", "Device 3"]
    levels = ["Low", "Medium", "High"]
    wavelengths = ["Red", "Green", "Blue"]
    durations = ["Short", "Medium", "Long"]
    
    create_buttons_with_dropdowns(main_frame, devices, levels, wavelengths, durations)
    
    text_box = Text(root, bg="black", fg="white", width=40, height=10, state=DISABLED)
    text_box.pack(side=RIGHT, padx=10, pady=10)

    root.mainloop()


In [3]:
def filter_and_save_excel(input_csv = 'data/main.csv', output_excel = 'data/output.xlsx'):
    # Check if the input CSV file exists
    if not os.path.exists(input_csv):
        # If the file doesn't exist, create an Excel file with an error message
        with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
            df_error = pd.DataFrame({'Error': ['Input CSV file not found']})
            df_error.to_excel(writer, sheet_name='Error', index=False)
        return
    
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Filter columns where the header contains "L1" or "L5"
    l1_columns = df.filter(regex=r'\bL1\b', axis=1)
    l5_columns = df.filter(regex=r'\bL5\b', axis=1)
    
    # Sort the columns alphabetically based on the column names
    l1_columns = l1_columns.reindex(sorted(l1_columns.columns), axis=1)
    l5_columns = l5_columns.reindex(sorted(l5_columns.columns), axis=1)
    
    # Create a new Excel writer object
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        # Write the filtered and sorted data to different sheets
        if not l1_columns.empty:
            l1_columns.to_excel(writer, sheet_name='L1 Columns', index=False)
        if not l5_columns.empty:
            l5_columns.to_excel(writer, sheet_name='L5 Columns', index=False)
        
        # If both are empty, add a default sheet to avoid the IndexError
        if l1_columns.empty and l5_columns.empty:
            df_empty = pd.DataFrame({'Message': ['No columns with L1 or L5 found']})
            df_empty.to_excel(writer, sheet_name='Info', index=False)

In [ ]:





root = Tk()
root.title("Device Stability Testing")
root.geometry("810x490")


text_box = Text(root, bg ='black', fg = 'white', height=10, width=100, state = DISABLED)
text_box.pack(except = True, fill = X, padx = 10, pady = 10)


button_list = ["b1","b2","b3","b4","b5","b6","b7","b8","b9","b0"]
# Add the filter, based on the button_list wavelength will be decided
device_config = {}

for button in button_list:
    if "bv" in button:
        device_config[button][0] = ["528","620"]
        device_config[button][1] = ["L1","L5"]
        
    elif "duo" in button:
        device_config[button][0] = ["528","620","367"]
        device_config[button][1] = ["L1","L5"]
        
rm_wavelength_list, rm_level_list = gui(f1,device_config,button_list)

# remove the wavelength and level from the device_config
for button in button_list:
    for rm_wavelength in rm_wavelength_list:
        if rm_wavelength in device_config[button][0]:
            device_config[button][0].remove(rm_wavelength)
    for rm_level in rm_level_list:
        if rm_level in device_config[button][1]:
            device_config[button][1].remove(rm_level)
            